## EXAMPLE Quadtree

In [ ]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd

from hydromt_sfincs import SfincsModel
from hydromt_sfincs import utils

In [ ]:
# Initialize SfincsModel Python class with the artifact data catalog which contains publically available data for North Italy
# we overwrite (mode='w+') the existing model in the root directory if it exists
sf = SfincsModel(data_libs=["artifact_data"], root="test_quadtree", mode="w+")

In [ ]:
# Specify an input dictionary with the grid settings x0,y0,dx,dy,nmax,mmax,rotation and epsg code.
# create SFINCS model with regular grid and characteristics of the input dictionary:
sf.setup_grid(
    x0=318650,
    y0=5040000,
    dx=50.0,
    dy=50.0,
    nmax=107,
    mmax=250,
    rotation=27,
    epsg=32633,
)

In [ ]:
# show the model grid outline
# sf.region.boundary.plot(figsize=(6,6))
_ = sf.plot_basemap(plot_region=True, bmap="sat", zoomlevel=12)

In [ ]:
# now create a geodataframe that covers a part of the model grid
datasets_dep = [{"elevtn": "merit_hydro", "zmin": 0.001}, {"elevtn": "gebco"}]

# Add depth information to modelgrid based on these chosen datasets
sf.setup_dep(datasets_dep=datasets_dep)
sf.setup_mask_active(zmin=-5, reset_mask=True)
sf.setup_mask_bounds(btype="waterlevel", zmax=-5, reset_bounds=True)


In [ ]:
# derive river from hydrography data based on a minimum river length (river_len) 
# and minimum upstream area (river_upa)

sf.setup_river_inflow(
    hydrography='merit_hydro',
    river_len=1000,
    river_upa=50, 
    keep_rivers_geom=True
)


In [ ]:
gdf_riv = sf.geoms['rivers_inflow'].copy()
gdf_riv_buf = gdf_riv.assign(geometry=gdf_riv.buffer(200))
gdf_riv_buf["refinement_level"] = 2

# Make a plot of model
# note the src points and derived river network
fig, ax = sf.plot_basemap(variable="dep", plot_bounds=False, bmap="sat")

# plot gdf_riv on top
gdf_riv_buf.plot(ax=ax, color="red", linewidth=0.5)

In [ ]:
import importlib

from hydromt_sfincs import quadtree

# NOTE this is to not have to restart the kernel all the time
importlib.reload(quadtree)

# create a quadtree object
qt = quadtree.QuadtreeGrid()

In [ ]:
qt.build(x0=318650, y0=5040000, dx=50.0, dy=50.0, nmax=107, mmax=250, rotation=27, epsg=32633,
         refinement_polygons=gdf_riv_buf)

In [ ]:
qt.data["level"].ugrid.plot()

## Uncomment for read/write

In [ ]:
# qt.write("sfincs2.nc")

# qt2 = quadtree.QuadtreeGrid()
# qt2.read("sfincs2.nc")

In [ ]:
da = sf.data_catalog.get_rasterdataset("gebco", 
                        bbox=sf.mask.raster.transform_bounds(4326),
                        buffer=10,
                        variables=["elevtn"],
                        zoom_level=(sf.grid.raster.res, "meter"),)

qt.set_bathymetry(da_dep=da)

In [ ]:
da.plot()

In [ ]:
sf.grid["dep"].plot(x="xc", y="yc")

In [ ]:
qt.data["z"].ugrid.plot()

In [ ]:
# %matplotlib qt
%matplotlib inline

qt.set_mask(model="sfincs",zmin=-5, zmax=5)
qt.data["mask"].ugrid.plot()

In [ ]:
qt.set_mask(model="snapwave",zmin=-5, zmax=5, include_polygon=gdf_riv_buf) #> doesnt work yet because of expect z values in set_mask, I removed this criteria for now (/question whether we want this behaviour, that include_polygon doesn't overrule)
qt.data["snapwave_mask"].ugrid.plot()

In [ ]:
qt.write("sfincs_snapwave.nc")

Try situation where we want to directly copy the SFINCS mask to SnapWave:

In [ ]:
qt.set_mask(model="sfincs",zmin=-5, zmax=5, include_polygon=gdf_riv_buf) #> doesnt work yet because of expect z values in set_mask, I removed this criteria for now (/question whether we want this behaviour, that include_polygon doesn't overrule)
qt.data["mask"].ugrid.plot()

In [ ]:
qt.set_mask(model="snapwave", copy_sfincs_mask2snapwave=True)
qt.data["snapwave_mask"].ugrid.plot()

In [ ]:
qt.write("sfincs_snapwave_v2.nc")